In [2]:
import fasttext
import numpy as np
import json
import random
import datetime
import os

Via https://fasttext.cc

```
wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.it.300.bin.gz
wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.it.zip
```

In [6]:
wikimodel = fasttext.FastText.load_model("data/wiki.it/wiki.it.bin")

ccmodel = fasttext.FastText.load_model("data/cc.it.300.bin")

In [7]:
def cosine_distance(first, second, model, decimals=2):
    exact_distance = (np.dot(model[first], model[second])
            /(np.linalg.norm(model[first])*np.linalg.norm(model[second]))
           )
    truncated_distance = np.round(float(exact_distance*100),decimals)
    return truncated_distance

## Extract Apple dictionary

https://gist.github.com/josephg/5e134adf70760ee7e49d

In [4]:
# Thanks to commenters for providing the base of this much nicer implementation!
# Save and run with $ python 0dedict.py
# You may need to hunt down the dictionary files yourself and change the awful path string below.
# This works for me on MacOS 10.14 Mohave

from struct import unpack
from zlib import decompress
import re
# filename = '/System/Library/Assets/com_apple_MobileAsset_DictionaryServices_dictionaryOSX/9f5862030e8f00af171924ebbc23ebfd6e91af78.asset/AssetData/Oxford Dictionary of English.dictionary/Contents/Resources/Body.data'
# filename = "/System/Library/AssetsV2/com_apple_MobileAsset_DictionaryServices_dictionaryOSX/bad4a5c5bf10a2dd135a07d7ad7b28e23d0aa7dd.asset/AssetData/Italian.dictionary/Contents/Resources/Body.data"
filename = "/System/Library/AssetsV2/com_apple_MobileAsset_DictionaryServices_dictionaryOSX/a1fa9ccfe47b097a31a6b2ea94641e6e511855dd.asset/AssetData/Italian.dictionary/Contents/Resources/Body.data"

f = open(filename, 'rb')

def gen_entry():
    f.seek(0x40)
    limit = 0x40 + unpack('i', f.read(4))[0]
    f.seek(0x60)
    while f.tell()<limit:
        sz, = unpack('i', f.read(4))
        buf = decompress(f.read(sz)[8:])

        pos = 0
        while pos < len(buf):
            chunksize, = unpack('i', buf[pos:pos+4])
            pos += 4

            entry = buf[pos:pos+chunksize]
            title = re.search('d:title="(.*?)"', entry.decode('utf-8')).group(1)
            yield title, entry

            pos += chunksize

wordlist = []
for word, definition in gen_entry():
    wordlist.append(word)

In [5]:
print(len(wordlist))

96859


# Parole italiane

```
git clone https://github.com/napolux/paroleitaliane.git

sudo apt-get install witalian

/usr/share/dict/italian
```

In [6]:
parole60  = "data/paroleitaliane/paroleitaliane/60000_parole_italiane.txt"
parole95  = "data/paroleitaliane/paroleitaliane/95000_parole_italiane_con_nomi_propri.txt"

parole280 = "data/paroleitaliane/paroleitaliane/280000_parole_italiane.txt"

paroleDebian = "data/italian"

# with open(paroleDebian, "r") as file:
#     words = [w.strip() for w in file.readlines() if "'" not in w]
    


In [8]:
words = [w for w in wordlist if len(w)>1 and '-' not in w and '(' not in w and ')' not in w and "'" not in w]

In [9]:
words

['abaca',
 'abacà',
 'abaco',
 'abadessa',
 'abadia',
 'abalietà',
 'ab antiquo',
 'abarico',
 'abasia',
 'abate',
 'abatino',
 'abaton',
 'abaya',
 'abazia',
 'abaziale',
 'abbacare',
 'abbacchiamento',
 'abbacchiare',
 'abbacchiato',
 'abbacchiatura',
 'abbacchio',
 'abbacinamento',
 'abbacinante',
 'abbacinare',
 'abbaco',
 'abbadare',
 'abbadessa',
 'abbadia',
 'abbagliamento',
 'abbagliante',
 'abbagliare',
 'abbaglio',
 'abbaiare',
 'abbaiata',
 'abbaino',
 'abbaio',
 'abbaio',
 'abballare',
 'abballottare',
 'abbambinare',
 'abbambolato',
 'abbancare',
 'abbandonare',
 'abbandonato',
 'abbandonico',
 'abbandonismo',
 'abbandono',
 'abbarbagliamento',
 'abbarbagliare',
 'abbarbaglio',
 'abbarbaglio',
 'abbarbicamento',
 'abbarbicante',
 'abbarbicare',
 'abbarcare',
 'abbarcare',
 'abbarcatura',
 'abbarrare',
 'abbaruffare',
 'abbaruffio',
 'abbassabile',
 'abbassalingua',
 'abbassamento',
 'abbassare',
 'abbassata',
 'abbasso',
 'abbastanza',
 'abbate',
 'abbattere',
 'abbattibil

In [10]:
len(words)

95264

In [11]:
with open("data/paroleitaliane/paroleitaliane/1000_parole_italiane_comuni.txt", "r") as file:
    common_words = [w.strip() for w in file.readlines()]
    
common_words = [w for w in common_words if len(w)>2]
len(common_words)

1132

In [12]:
def create_semantle_data(all_words, solution_words, model, date, puzzle_number, verbose=False):
    # Day to use for the solution
    isoday = date.isoformat()
    
    # Solution
    # Seed the random number generator
    random.seed(a=isoday, version=2)
    # Pick the word
    query = random.choice(solution_words)
    if verbose:
        print("La soluzione per il {} è \"{}\"".format(isoday, query))
        
    # Distanze
    distances_list = [(word, cosine_distance(query, word, model)) for word in set(all_words+solution_words) if len(word)>1]
    
    # Distanze nell'ordine giusto
    distances_list_sorted = sorted(distances_list, key=lambda x: x[1],reverse=True)
    
    # Database per Semantle
    
    word_database = {
        _tuple[0]:
        {
            "s": _tuple[1],
            "r": _ix,
        } for _ix, _tuple in enumerate(distances_list_sorted)
    }
    
    # Le parole più vicine
    closest_words_list = [
        {
            "w": _tuple[0],
            "s": _tuple[1],
            "r": _ix,
        } for _ix, _tuple in enumerate(distances_list_sorted[:1000])
    ]
    
    solution_word = query
    
    # Statistiche
    
    nearest_word_similarity = closest_words_list[1]["s"]
    tenth_nearest_word_similarity = closest_words_list[9]["s"]
    thousandth_nearest_word_similarity = closest_words_list[999]["s"]

    day_stats = {
        "puzzle_number": puzzle_number,
        "nearest_word_similarity": nearest_word_similarity,
        "tenth_nearest_word_similarity": tenth_nearest_word_similarity,
        "thousandth_nearest_word_similarity": thousandth_nearest_word_similarity,
    }

    semantle_data = {
        "word_database": word_database,
        "closest_words_list": closest_words_list,
        "solution_word": solution_word,
        "day_stats": day_stats,
    }
    return semantle_data

In [13]:
wikidata = create_semantle_data(
    all_words=words,
    solution_words=common_words,
    model=wikimodel,
    date=datetime.date.today()-datetime.timedelta(days=2),
    puzzle_number=-1,
    verbose=True
)

ccdata = create_semantle_data(
    all_words=words,
    solution_words=common_words,
    model=ccmodel,
    date=datetime.date.today()-datetime.timedelta(days=2),
    puzzle_number=-1,
    verbose=True
)

La soluzione per il 2023-06-08 è "lei"
La soluzione per il 2023-06-08 è "lei"


In [14]:
print([w['w'] for w in wikidata['closest_words_list'][:50]])
# print(wikidata['word_database']['re'])

['lei', 'lui', 'ragazza', 'fidanzato', 'innamorato', 'arrabbiata', 'ragazzo', 'incinta', 'ella', 'marito', 'bambina', 'sorella', 'fidanzare', 'confidanza', 'innamorare', 'donna', 'invaghito', 'matrigna', 'risposare', 'follemente', 'ricambiare', 'ingelosire', 'infatuato', 'ragazzata', 'bambinaia', 'infatuare', 'flirtare', 'contraccambiare', 'matrignesco', 'perdutamente', 'madre', 'moglie', 'baciare', 'baciato', 'colei', 'invaghire', 'nonna', 'litigata', 'geloso', 'sorellastra', 'arrabbiato', 'spasimante', 'disinnamorare', 'matrignare', 'segretamente', 'nuora', 'sposare', 'sentimentalmente', 'costei', 'infatuazione']


In [15]:
print([w['w'] for w in ccdata['closest_words_list'][:50]])
# print(ccdata['word_database']['re'])

['lei', 'lui', 'ella', 'ragazza', 'donna', 'colei', 'perché', 'sorella', 'marito', 'figlia', 'signora', 'costei', 'bambina', 'moglie', 'qualcuno', 'madre', 'signorina', 'che', 'fidanzato', 'glielo', 'intanto', 'compagna', 'siccome', 'chi', 'però', 'egli', 'invece', 'adesso', 'naturalmente', 'poi', 'quando', 'allora', 'mamma', 'insomma', 'ovviamente', 'nessuno', 'fida', 'persona', 'pure', 'evidentemente', 'dica', 'poveraccia', 'nonna', 'come', 'non', 'quale', 'noi', 'soltanto', 'anche', 'costui']


In [16]:
data = create_semantle_data(
    all_words=words,
    solution_words=common_words,
    model=wikimodel,
    date=datetime.date(2022,9,11),
    puzzle_number=1,
    verbose=True
)

La soluzione per il 2022-09-11 è "nobile"


In [17]:
data = create_semantle_data(
    all_words=words,
    solution_words=common_words,
    model=ccmodel,
    date=datetime.date(2022,9,11),
    puzzle_number=1,
    verbose=True
)
print(data["word_database"]["conte"])
print([n["w"] for n in data["closest_words_list"]])


La soluzione per il 2022-09-11 è "nobile"
{'s': 47.61, 'r': 21}
['nobile', 'nobiltà', 'nobiliare', 'aristocratico', 'nobildonna', 'nobiluomo', 'casato', 'illustre', 'patrizio', 'casata', 'aristocrazia', 'nobilitato', 'umile', 'nobilitare', 'gentiluomo', 'benestante', 'discendente', 'ignobile', 'famiglia', 'eminente', 'amabile', 'conte', 'principe', 'cortigiano', 'possidente', 'stirpe', 'plebeo', 'lignaggio', 'magnatizio', 'gentildonna', 'marchese', 'nobilitazione', 'cavaliere', 'barone', 'contessa', 'duca', 'rampollo', 'antico', 'notabile', 'decurionale', 'virile', 'facoltoso', 'rispettabile', 'prosapia', 'baronale', 'signorotto', 'marchesana', 'lodevole', 'ceto', 'vile', 'blasone', 'feudatario', 'giovane', 'regnante', 'schiatta', 'caritatevole', 'patriotta', 'prelato', 'rango', 'generoso', 'cinquecentesco', 'genealogista', 'regale', 'potente', 'decaduto', 'letterato', 'umanista', 'marchionale', 'temperante', 'rinascimentale', 'condottiere', 'filantropo', 'magnanimo', 'valoroso', 'cava

In [18]:
for days_ahead in range(365):
    _date = datetime.date(2022,9,9)+datetime.timedelta(days=days_ahead)
    data = create_semantle_data(
        all_words=words,
        solution_words=common_words,
        model=ccmodel,
        date=_date,
        puzzle_number=days_ahead,
        verbose=True
    )
    isopath = _date.isoformat()
    os.makedirs("AppleCC/"+isopath, exist_ok = True)
    with open("AppleCC/"+isopath+"/semantle.json", "w") as file:
        json.dump(data,file)
    with open("AppleCC/"+isopath+"/closest.json", "w") as file:
        json.dump(data['closest_words_list'],file)
    with open("AppleCC/"+isopath+"/stats.json", "w") as file:
        json.dump(data['day_stats'],file)
    with open("AppleCC/"+isopath+"/database.json", "w") as file:
        json.dump(data['word_database'],file)

La soluzione per il 2022-09-09 è "zio"
La soluzione per il 2022-09-10 è "qualsiasi"
La soluzione per il 2022-09-11 è "nobile"
La soluzione per il 2022-09-12 è "segno"
La soluzione per il 2022-09-13 è "correre"
La soluzione per il 2022-09-14 è "insomma"
La soluzione per il 2022-09-15 è "caratteristico"
La soluzione per il 2022-09-16 è "accompagnare"
La soluzione per il 2022-09-17 è "settimana"
La soluzione per il 2022-09-18 è "rapporto"
La soluzione per il 2022-09-19 è "colore"
La soluzione per il 2022-09-20 è "regione"
La soluzione per il 2022-09-21 è "spesa"
La soluzione per il 2022-09-22 è "realtà"
La soluzione per il 2022-09-23 è "minuto"
La soluzione per il 2022-09-24 è "giornale"
La soluzione per il 2022-09-25 è "entro"
La soluzione per il 2022-09-26 è "necessità"
La soluzione per il 2022-09-27 è "viaggio"
La soluzione per il 2022-09-28 è "tuttavia"
La soluzione per il 2022-09-29 è "capitare"
La soluzione per il 2022-09-30 è "accorgersi"
La soluzione per il 2022-10-01 è "oggi"
La 

La soluzione per il 2023-03-19 è "muovere"
La soluzione per il 2023-03-20 è "mandare"
La soluzione per il 2023-03-21 è "periodo"
La soluzione per il 2023-03-22 è "vita"
La soluzione per il 2023-03-23 è "studiare"
La soluzione per il 2023-03-24 è "strano"
La soluzione per il 2023-03-25 è "leggero"
La soluzione per il 2023-03-26 è "sala"
La soluzione per il 2023-03-27 è "suo"
La soluzione per il 2023-03-28 è "giungere"
La soluzione per il 2023-03-29 è "limitare"
La soluzione per il 2023-03-30 è "mangiare"
La soluzione per il 2023-03-31 è "segreto"
La soluzione per il 2023-04-01 è "presso"
La soluzione per il 2023-04-02 è "figlio"
La soluzione per il 2023-04-03 è "diventare"
La soluzione per il 2023-04-04 è "movimento"
La soluzione per il 2023-04-05 è "principe"
La soluzione per il 2023-04-06 è "aiuto"
La soluzione per il 2023-04-07 è "eccellenza"
La soluzione per il 2023-04-08 è "sorprendere"
La soluzione per il 2023-04-09 è "ottenere"
La soluzione per il 2023-04-10 è "direttore"
La solu

In [76]:
wikidata.keys()

dict_keys(['word_database', 'closest_words_list', 'solution_word', 'day_stats'])